# SDV Exploration
Looking into the open source [SDV library](https://github.com/sdv-dev/SDV) to see if it could be a good option going forward for our use case.

Before running this notebook, go ahead and `pip install sdv` so you have access to the package!

In [112]:
import sdv
import pandas as pd
import datetime as dt
import numpy as np
from pandas_profiling import ProfileReport

pd.set_option('display.max_columns', None)

### Import Data

This notebook makes use of [this covid dataset](https://github.com/beoutbreakprepared/nCoV2019/blob/master/latest_data/latestdata.tar.gz). It's too big to store in github easily, so download it locally and update the path below as needed to point to where you've put it.

In [119]:
PATH_TO_DATA = 'latestdata.csv'
USE_WHOLE_DATASET = False # Uses whole dataset if True, first 1000 points otherwise. Latter is a lot faster!

### Create Tables
SDV allows you to import several datasets together and link via primary keys. The tables are organized into a dictionary, where the dict keys are the table names and the dict values are the tables stored as dataframes. We only have one table we care about in this use case, but we might want to make use of this several table structure in the future!

In [114]:
# Prep to parse dates on read
custom_date_parser = lambda x: pd.to_datetime(x, format='%d.%m.%Y', errors='coerce')
date_columns = ['date_confirmation',
                'date_onset_symptoms',
                'date_admission_hospital',
                'travel_history_dates',
                'date_death_or_discharge']


# Read covid data csv, parsing dates as datetimes rather than strings
df = pd.read_csv(PATH_TO_DATA, parse_dates=date_columns, date_parser=custom_date_parser)
df.head()

/Users/skyler/.pyenv/versions/3.7.2/envs/kind/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1,2,12,13,15,16,17,19,20,21,22,24,25,26,27,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,age,sex,city,province,country,latitude,longitude,geo_resolution,date_onset_symptoms,date_admission_hospital,date_confirmation,symptoms,lives_in_Wuhan,travel_history_dates,travel_history_location,reported_market_exposure,additional_information,chronic_disease_binary,chronic_disease,source,sequence_available,outcome,date_death_or_discharge,notes_for_discussion,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials,travel_history_binary
0,000-1-1,NaN,male,Shek Lei,Hong Kong,China,22.365019,114.133808,point,NaT,NaT,2020-02-14,NaN,NaN,2020-01-22,China,NaN,Case 55; mainland China travel via the Lok Ma ...,False,NaN,https://www.scmp.com/news/hong-kong/health-env...,NaN,"critical condition, intubated as of 14.02.2020",NaT,NaN,Shek Lei,NaN,NaN,Hong Kong,China,8029.0,NaN,NaN
1,000-1-10,78,male,Vo Euganeo,Veneto,Italy,45.297748,11.658382,point,NaT,NaT,2020-02-21,NaN,NaN,NaT,NaN,NaN,Hospitalized on 12.02.2020 for other reasons,False,NaN,https://www.corriere.it/cronache/20_febbraio_2...,NaN,death,2020-02-22,NaN,Vo' Euganeo,NaN,NaN,Veneto,Italy,8954.0,NaN,NaN
2,000-1-100,61,female,NaN,NaN,Singapore,1.353460,103.815100,admin0,NaT,NaT,2020-02-14,NaN,NaN,NaT,NaN,NaN,"Case 65; family member of Case 50, a DBS emplo...",False,NaN,https://www.channelnewsasia.com/news/singapore...,NaN,discharge,2020-02-17,NaN,NaN,NaN,NaN,NaN,Singapore,200.0,NaN,NaN
3,000-1-1000,NaN,NaN,Zhengzhou City,Henan,China,34.629310,113.468000,admin2,NaT,NaT,2020-01-26,NaN,NaN,NaT,NaN,NaN,NaN,False,NaN,https://news.163.com/special/epidemic/?spssid=...,NaN,NaN,NaT,NaN,NaN,NaN,Zhengzhou City,Henan,China,10091.0,NaN,NaN
4,000-1-10000,NaN,NaN,Pingxiang City,Jiangxi,China,27.513560,113.902900,admin2,NaT,NaT,2020-02-14,NaN,NaN,NaT,NaN,NaN,NaN,False,NaN,http://hc.jiangxi.gov.cn/doc/2020/02/15/138898...,NaN,NaN,NaT,NaN,NaN,NaN,Pingxiang City,Jiangxi,China,7060.0,NaN,NaN


In [92]:
len(df)

2676311

On recommendations from Gretel AI and team discussion, I'm dropping the source url column. We may also want to shorten lat and long to 3 or 4 digits of precision. 

In [116]:
df = df.drop(columns=['source'])

In [139]:
demo_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ID                        10000 non-null  object        
 1   age                       902 non-null    object        
 2   sex                       1028 non-null   object        
 3   city                      4060 non-null   object        
 4   province                  9116 non-null   object        
 5   country                   9985 non-null   object        
 6   latitude                  9998 non-null   float64       
 7   longitude                 9998 non-null   float64       
 8   geo_resolution            9998 non-null   object        
 9   date_onset_symptoms       307 non-null    datetime64[ns]
 10  date_admission_hospital   267 non-null    datetime64[ns]
 11  date_confirmation         9966 non-null   datetime64[ns]
 12  symptoms           

This dataset is pretty big. For speed purposes, I'm sometimes only grabbing the first 1000 rows for now so that this trains more quickly and I can test more rapidly.

In [122]:
if USE_WHOLE_DATASET:
    demo_df = df.copy()
else:
    demo_df = df.head(10000).copy()
tables = {'cases': demo_df}

In [121]:
original_profile = ProfileReport(demo_df)
original_profile.to_widgets()

/Users/skyler/.pyenv/versions/3.7.2/envs/kind/lib/python3.7/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Summarize dataset:   0%|          | 0/45 [00:00<?, ?it/s]

/Users/skyler/.pyenv/versions/3.7.2/envs/kind/lib/python3.7/site-packages/pandas_profiling/model/correlations.py:139: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  (include the error message: '{error}')"""


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### Create Metadata
[Metadata Documentation](https://sdv.dev/SDV/user_guides/relational/relational_metadata.html)

In this first pass, we won't add any constraints to our model.

In [123]:
metadata = sdv.Metadata()
metadata.add_table(
    name='cases', 
    data=tables['cases'], 
    primary_key='ID')

In [124]:
metadata

Metadata
  root_path: .
  tables: ['cases']
  relationships:

### Fit SDV (Without Constraints)
This fits a model that you can then sample from to generate fake data. The more data we use as input, the longer this takes -- using the whole 2 million row dataset will take 20+ minutes.

In [125]:
svd_model = sdv.SDV()

In [126]:
svd_model.fit(metadata, tables)

In [127]:
svd_model.save('sdv_no_constraints.pkl')

### Sample SDV (Without Constraints)
After creating the model, we can then sample points from it to create a fake dataset. The sample() default is to create a dataset that's of the same size as the set it trained on, but you can also set the number of samples you want to generate.

In [128]:
svd_model = sdv.SDV.load('sdv_no_constraints.pkl')

In [129]:
samples = svd_model.sample(num_rows=10000)

In [140]:
samples_df = samples['cases']
samples_df.head()

,ID,age,sex,city,province,country,latitude,longitude,geo_resolution,date_onset_symptoms,date_admission_hospital,date_confirmation,symptoms,lives_in_Wuhan,travel_history_dates,travel_history_location,reported_market_exposure,additional_information,chronic_disease_binary,chronic_disease,sequence_available,outcome,date_death_or_discharge,notes_for_discussion,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials,travel_history_binary
0,a,NaN,NaN,Nanning City,Kanagawa Prefecture,China,35.717291,70.283748,admin2,NaT,NaT,2020-02-24,NaN,NaN,NaT,NaN,NaN,NaN,False,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Comunidad de Madrid,China,4394.134560,NaN,NaN
1,b,NaN,NaN,Zhengzhou City,NaN,Spain,42.099323,40.843676,admin1,NaT,NaT,2020-03-05,NaN,NaN,NaT,NaN,NaN,NaN,False,NaN,NaN,NaN,NaT,NaN,Jaipur,NaN,NaN,Qom,China,6796.961008,NaN,NaN
2,c,NaN,NaN,Huainan City,East Azarbaijan,Italy,37.229407,182.951429,admin2,NaT,NaT,2020-01-30,NaN,NaN,NaT,NaN,NaN,NaN,False,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Anhui,Italy,9218.123222,NaN,NaN
3,d,NaN,NaN,Wenzhou City,NaN,China,42.106511,32.323230,admin2,NaT,NaT,2020-02-27,NaN,NaN,NaT,NaN,NaN,With cases subtracted from here http://www.sal...,False,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Qom,China,4948.502797,NaN,NaN
4,e,NaN,NaN,Nanning City,Tehran,Italy,26.418875,38.749249,admin1,NaT,NaT,2020-02-21,NaN,NaN,NaT,NaN,NaN,NaN,False,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Tehran,South Korea,11287.050260,NaN,NaN


### Evaluation
[Evaluation Documentation](https://github.com/sdv-dev/SDV/blob/master/EVALUATION.md)

**Score**: According to the docs, the output will be a maximization score that will indicate "how good" the modeling was: the higher the value, the more similar the sets of table are. Notice that in most cases the value will be negative. We'll have to go through the literature to understand what this score means... but it is likely useful once we understand it?

Calculating this score is somewhat computationally intensive as well.

In [ ]:
score = sdv.evaluation.evaluate(samples, tables, metadata)
print("Eval score: ", score)

We can also do our own evaluation, starting by a visual check of how this dataset profile compares to our original.

A clear problem comes up looking at the city/country combinations: with this current sampling structure, we're ending up with cities that don't match the country correctly (ex. city=Zhengzhou City, country=Spain). We will need to fix that!

In [142]:
# Dropping columns that have all nan values in the profile
# Profiling tool seems to error when it gets all nan input cols
synthetic_profile = ProfileReport(samples_df.dropna(axis='columns', how='all'))
synthetic_profile.to_widgets()

/Users/skyler/.pyenv/versions/3.7.2/envs/kind/lib/python3.7/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Summarize dataset:   0%|          | 0/40 [00:00<?, ?it/s]

/Users/skyler/.pyenv/versions/3.7.2/envs/kind/lib/python3.7/site-packages/pandas_profiling/model/correlations.py:139: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  (include the error message: '{error}')"""


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### Using Constraints

[Constraints](https://github.com/sdv-dev/SDV/blob/master/tutorials/single_table_data/05_Handling_Constraints.ipynb) are ways of limiting the ways the data can be combined or saying values must be generated within specific ranges or using specific relationships. This will likely be useful across a variety of columns!

For example, we probably want to constrain the model to keep cities, provinces and country relationships the same rather than creating new combinations of the three. We can do that here using a `UniqueCombinations` constraint.

In [148]:
# Create constraints
unique_country_province_city_combos = sdv.constraints.UniqueCombinations(
    columns=['city','country'],
    handling_strategy='transform'
)
constraints = [unique_country_province_city_combos] # Could add more constraints later!

# Define a model using those constraints
gc = sdv.tabular.GaussianCopula(constraints=constraints)

In [149]:
gc.fit(demo_df.head(100))
sampled = gc.sample(10)

/Users/skyler/.pyenv/versions/3.7.2/envs/kind/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:621: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Users/skyler/.pyenv/versions/3.7.2/envs/kind/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The number of calls to function has reached maxfev = 600.
  warnings.warn(msg, RuntimeWarning)
/Users/skyler/.pyenv/versions/3.7.2/envs/kind/lib/python3.7/site-packages/scipy/optimize/optimize.py:734: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):


In [151]:
sampled.head()

,ID,age,sex,city,province,country,latitude,longitude,geo_resolution,date_onset_symptoms,date_admission_hospital,date_confirmation,symptoms,lives_in_Wuhan,travel_history_dates,travel_history_location,reported_market_exposure,additional_information,chronic_disease_binary,chronic_disease,sequence_available,outcome,date_death_or_discharge,notes_for_discussion,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials,travel_history_binary
0,000-1-10062,NaN,male,Zhengzhou City,Jilin,China,32.190099,133.160821,admin2,NaT,NaT,2020-01-30,NaN,NaN,NaT,NaN,NaN,Close contacts of confirmed cases in Panjin Ci...,False,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Zhengzhou City,Jilin,China,6999.036261,NaN,NaN
1,000-1-1007,NaN,NaN,Zhengzhou City,Veneto,China,38.770782,124.473477,admin2,NaT,NaT,2020-01-22,NaN,NaN,NaT,NaN,NaN,NaN,False,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Zhengzhou City,Veneto,China,9363.645883,NaN,NaN
2,000-1-10065,NaN,NaN,"Gongzhuling City, Siping City",Jiangxi,China,31.397488,114.977399,admin2,NaT,NaT,2020-02-10,NaN,NaN,NaT,NaN,NaN,Case 55; mainland China travel via the Lok Ma ...,False,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Jiangxi,China,9415.709833,NaN,NaN
3,000-1-10056,NaN,NaN,Yichun City,Jiangxi,China,33.360961,129.442757,admin2,NaT,NaT,2020-02-05,NaN,NaN,NaT,NaN,NaN,NaN,False,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,Jiangxi,China,7901.456679,NaN,NaN
4,000-1-1000,NaN,NaN,Zhengzhou City,Jiangxi,China,25.371432,114.168992,admin2,NaT,NaT,2020-02-06,NaN,NaN,NaT,NaN,NaN,NaN,False,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Zhengzhou City,Jiangxi,China,10018.534583,NaN,NaN
